<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: COVID-19 Dashboard
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Análise Exploratória de Dados;</li>
  <li>Visualização Interativa de Dados;</li>
  <li>Storytelling.</li>
</ol>


---

# **Exercícios**

Este *notebook* deve servir como um guia para **você continuar** a construção da sua própria análise exploratória de dados interativa. Fique a vontate para copiar os códigos da aula mas busque explorar os dados ao máximo. Por fim, publique seu *notebook* no [Kaggle](https://www.kaggle.com/) e seu *dashboard* [Google Data Studio](https://datastudio.google.com/).

---

# **COVID Dashboard**

## 1\. Contexto

A **COVID-19**, causada pelo coronavírus SARS-CoV-2, é uma infecção respiratória aguda de elevada transmissibilidade e potencial gravidade, que se espalhou globalmente. A obtenção e análise de dados sobre a evolução da pandemia são essenciais para o enfrentamento eficaz em cada região. Este projeto visa desenvolver um dashboard interativo para explorar e visualizar os dados relacionados ao avanço de casos e à vacinação no Brasil. O objetivo é permitir um comparativo entre o Brasil e outras regiões do mundo, fornecendo insights sobre a disseminação dos casos e o progresso da vacinação.

**1.1. TLDR**

* Dashboard:

    * Google Data Studio [Link](https://lookerstudio.google.com/reporting/12b5bd36-3079-4806-8a84-8321b3b7add5)

* Processamento:

    * Kaggle Notebook [Link](https://www.kaggle.com/code/rodrigoflores78/covid-19)

* Fontes:
    * Casos pela universidade John Hopkins [Link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports)
    * Vacinação pela universidade de Oxford [Link](https://www.google.com/url?q=https%3A%2F%2Fcovid.ourworldindata.org%2Fdata%2Fowid-covid-data.csv)


## 2\. Pacotes e bibliotecas

In [92]:
import math
from typing import Iterator
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

## 3\. Extração **Casos**

In [93]:
cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-12-2021.csv', sep=',')

In [94]:
south_american_countries = [
    'Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador',
    'Guyana', 'Paraguay', 'Peru', 'Suriname', 'Uruguay', 'Venezuela'
]

In [95]:
def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
    date_range_days: int = (end_date - start_date).days
    for lag in range(date_range_days):
        yield start_date + timedelta(lag)

In [96]:
start_date = datetime(2021,  1,  1)
end_date   = datetime(2021, 12, 31)

In [97]:
cases = None
cases_is_empty = True


for date in date_range(start_date=start_date, end_date=end_date):
    date_str = date.strftime('%m-%d-%Y')
    data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'


    try:
        case = pd.read_csv(data_source_url, sep=',')
    except Exception as e:
        print(f"Erro ao carregar os dados de {date_str}: {e}")
        continue


    case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio', 'Province_State'], axis=1, errors='ignore')


    case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))


    case['Region'] = case['Country_Region'].apply(
        lambda x: 'Brazil' if x == 'Brazil' else 'South America' if x in south_american_countries else None
    )

    if cases_is_empty:
        cases = case
        cases_is_empty = False
    else:
        cases = pd.concat([cases, case], ignore_index=True)

In [98]:
cases.head()

,Country_Region,Confirmed,Deaths,Incident_Rate,Date,Region
0,Afghanistan,52513,2201,134.896578,2021-01-01,None
1,Albania,58316,1181,2026.409062,2021-01-01,None
2,Algeria,99897,2762,227.809861,2021-01-01,None
3,Andorra,8117,84,10505.403482,2021-01-01,None
4,Angola,17568,405,53.452981,2021-01-01,None


In [99]:
cases.shape

(1461252, 6)

In [100]:
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461252 entries, 0 to 1461251
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   Country_Region  1461252 non-null  object        
 1   Confirmed       1461252 non-null  int64         
 2   Deaths          1461252 non-null  int64         
 3   Incident_Rate   1428400 non-null  float64       
 4   Date            1461252 non-null  datetime64[ns]
 5   Region          40768 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 66.9+ MB


## 4\. Transformação **Casos**

In [101]:
cases = cases.rename(columns={'Country_Region': 'country'})
cases.columns = cases.columns.str.lower()

for col in cases.columns:
  cases = cases.rename(columns={col: col.lower()})

In [102]:
cases['month'] = cases['date'].apply(lambda date: date.strftime('%Y-%m'))
cases['year']  = cases['date'].apply(lambda date: date.strftime('%Y'))

In [103]:
cases['incident_rate'] = cases['incident_rate'].fillna(0)


cases['population'] = np.where(
    cases['incident_rate'] > 0,
    round(100000 * (cases['confirmed'] / cases['incident_rate'])),
    np.nan
)

cases = cases.drop('incident_rate', axis=1)

cases = cases[cases['region'].notna()]

In [104]:
cases.head()

,country,confirmed,deaths,date,region,month,year,population
6,Argentina,1629594,43319,2021-01-01,South America,2021-01,2021,45195777.0
38,Bolivia,160985,9175,2021-01-01,South America,2021-01,2021,11673029.0
41,Brazil,41689,796,2021-01-01,Brazil,2021-01,2021,881935.0
42,Brazil,105091,2496,2021-01-01,Brazil,2021-01,2021,3337357.0
43,Brazil,68361,926,2021-01-01,Brazil,2021-01,2021,845731.0


In [105]:
cases.shape

(40768, 8)

In [106]:
cases_ = None
cases_is_empty = True

def get_trend(rate: float) -> str:
    if np.isnan(rate):
        return np.NaN

    if rate < 0.75:
        status = 'downward'
    elif rate > 1.15:
        status = 'upward'
    else:
        status = 'stable'

    return status

for region in cases['region'].drop_duplicates():
    cases_per_region = cases.query(f'region == "{region}"').reset_index(drop=True)
    cases_per_region = cases_per_region.sort_values(by=['date'])

    cases_per_region['confirmed_1d'] = cases_per_region['confirmed'].diff(periods=1)
    cases_per_region['confirmed_moving_avg_7d'] = np.ceil(cases_per_region['confirmed_1d'].rolling(window=7).mean())
    cases_per_region['confirmed_moving_avg_7d_rate_14d'] = cases_per_region['confirmed_moving_avg_7d'] / cases_per_region['confirmed_moving_avg_7d'].shift(periods=14)
    cases_per_region['confirmed_trend'] = cases_per_region['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

    cases_per_region['deaths_1d'] = cases_per_region['deaths'].diff(periods=1)
    cases_per_region['deaths_moving_avg_7d'] = np.ceil(cases_per_region['deaths_1d'].rolling(window=7).mean())
    cases_per_region['deaths_moving_avg_7d_rate_14d'] = cases_per_region['deaths_moving_avg_7d'] / cases_per_region['deaths_moving_avg_7d'].shift(periods=14)
    cases_per_region['deaths_trend'] = cases_per_region['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

    if cases_is_empty:
        cases_ = cases_per_region
        cases_is_empty = False
    else:
        cases_ = pd.concat([cases_, cases_per_region], ignore_index=True)

cases = cases_
cases_ = None


In [107]:
cases.head()

/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,country,confirmed,deaths,date,region,month,year,population,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend
0,Argentina,1629594,43319,2021-01-01,South America,2021-01,2021,45195777.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Peru,25250,1606,2021-01-01,South America,2021-01,2021,1453700.0,-1604344.0,NaN,NaN,NaN,-41713.0,NaN,NaN,NaN
2,Peru,15304,835,2021-01-01,South America,2021-01,2021,668200.0,-9946.0,NaN,NaN,NaN,-771.0,NaN,NaN,NaN
3,Peru,48339,4261,2021-01-01,South America,2021-01,2021,1497400.0,33035.0,NaN,NaN,NaN,3426.0,NaN,NaN,NaN
4,Peru,7520,361,2021-01-01,South America,2021-01,2021,430700.0,-40819.0,NaN,NaN,NaN,-3900.0,NaN,NaN,NaN


In [108]:
cases['population'] = cases['population'].astype('Int64')
cases['confirmed_1d'] = cases['confirmed_1d'].astype('Int64')
cases['confirmed_moving_avg_7d'] = cases['confirmed_moving_avg_7d'].round(0).astype('Int64')
cases['deaths_1d'] = cases['deaths_1d'].astype('Int64')
cases['deaths_moving_avg_7d'] = cases['deaths_moving_avg_7d'].round(0).astype('Int64')

In [109]:
cases = cases[['date', 'country', 'region', 'population', 'confirmed', 'confirmed_1d', 'confirmed_moving_avg_7d', 'confirmed_moving_avg_7d_rate_14d', 'confirmed_trend', 'deaths', 'deaths_1d', 'deaths_moving_avg_7d', 'deaths_moving_avg_7d_rate_14d', 'deaths_trend', 'month', 'year']]

## 5\. Carregamento **Casos**

In [110]:
cases.to_csv('./covid-cases.csv', sep=',', index=False)

## 6\. Extração **Vacinas**

In [144]:
south_american_countries = [
    'Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador',
    'Guyana', 'Paraguay', 'Peru', 'Suriname', 'Uruguay', 'Venezuela'
]

In [145]:
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)

<ipython-input-145-608e6d2dd56d>:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)


In [146]:
vaccines['region'] = vaccines['location'].apply(
    lambda x: 'Brazil' if x == 'Brazil' else 'South America' if x in south_american_countries else 'Other'
)

vaccines = vaccines[vaccines['region'] != 'Other']

In [147]:
vaccines = vaccines[['location', 'region', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

In [148]:
vaccines.head()

,location,region,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
15066,Argentina,South America,45510324,NaN,NaN,NaN,NaN,2020-01-01
15067,Argentina,South America,45510324,NaN,NaN,NaN,NaN,2020-01-02
15068,Argentina,South America,45510324,NaN,NaN,NaN,NaN,2020-01-03
15069,Argentina,South America,45510324,NaN,NaN,NaN,NaN,2020-01-04
15070,Argentina,South America,45510324,NaN,NaN,NaN,NaN,2020-01-05


In [149]:
vaccines.shape

(20092, 8)

In [150]:
vaccines = vaccines.fillna(method='ffill')

<ipython-input-150-274988f553eb>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  vaccines = vaccines.fillna(method='ffill')


In [151]:
vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)

## 7\. Transformação **Vacinas**

In [152]:
vaccines = vaccines.rename(
  columns={
    'location': 'country',
    'total_vaccinations': 'total',
    'people_vaccinated': 'one_shot',
    'people_fully_vaccinated': 'two_shots',
    'total_boosters': 'three_shots',
  }
)

In [153]:
vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year']  = vaccines['date'].apply(lambda date: date.strftime('%Y'))

In [154]:
vaccines['one_shot_perc'] = round(vaccines['one_shot'] / vaccines['population'], 4)
vaccines['two_shots_perc'] = round(vaccines['two_shots'] / vaccines['population'], 4)
vaccines['three_shots_perc'] = round(vaccines['three_shots'] / vaccines['population'], 4)

In [155]:
vaccines['population'] = vaccines['population'].astype('Int64')
vaccines['total'] = vaccines['total'].astype('Int64')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int64')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int64')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int64')

In [156]:
vaccines = vaccines[['date', 'country', 'region', 'population', 'total', 'one_shot', 'one_shot_perc', 'two_shots', 'two_shots_perc', 'three_shots', 'three_shots_perc', 'month', 'year']]

In [157]:
vaccines.head()

,date,country,region,population,total,one_shot,one_shot_perc,two_shots,two_shots_perc,three_shots,three_shots_perc,month,year
0,2021-01-01,Argentina,South America,45510324,43532,43524,0.0010,7,0.0,1,0.0,2021-01,2021
1,2021-01-02,Argentina,South America,45510324,46844,46833,0.0010,10,0.0,1,0.0,2021-01,2021
2,2021-01-03,Argentina,South America,45510324,47290,47274,0.0010,11,0.0,5,0.0,2021-01,2021
3,2021-01-04,Argentina,South America,45510324,57753,57731,0.0013,15,0.0,7,0.0,2021-01,2021
4,2021-01-05,Argentina,South America,45510324,68484,68459,0.0015,18,0.0,7,0.0,2021-01,2021


## 8\. Carregamento **Vacinas**

In [158]:
vaccines.to_csv('./covid-vaccines.csv', sep=',', index=False)